[Home](../index.ipynb) / BlueTooth Sniffer
***
# BlueTooth Sniffer

---
## Source

In [ ]:
#%serialconnect --port=/dev/ttyUSB3 --baud=115200 
%serialconnect
import bluetooth
from micropython import const
from machine import Pin
import binascii
from time import ticks_ms
# Advertising payloads are repeated packets of the following form:
#   1 byte data length (N + 1)
#   1 byte type (see constants below)
#   N bytes type-specific data

mapScan  = {}
isLocked = False


class BLESniffer:
    def __init__( self ):
        self._ADV_TYPE_FLAGS              = const(0x01)
        self._ADV_TYPE_UUID16_INCOMPLETE  = const(0x02) # Incomplete List of 16-bit Service Class UUIDs
        self._ADV_TYPE_UUID16_COMPLETE    = const(0x03) # Complete List of 16-bit Service Class UUIDs
        self._ADV_TYPE_UUID32_INCOMPLETE  = const(0x04) # Incomplete List of 32-bit Service UUIDs
        self._ADV_TYPE_UUID32_COMPLETE    = const(0x05) # Complete List of 32-bit Service UUIDs
        self._ADV_TYPE_UUID128_INCOMPLETE = const(0x06) # Incomplete List of 128-bit Services
        self._ADV_TYPE_UUID128_COMPLETE   = const(0x07) # Complete List of 128-bit Services
        

        self._ADV_TYPE_SHORT_NAME       = const(0x08) # Shortened Local Name
        self._ADV_TYPE_NAME             = const(0x09) # Complete Local Name
        self._ADV_TYPE_EXTENSION        = const(0xFF) # Manufacturer extension
        
        self._IRQ_SCAN_RESULT = const(5)
        self._IRQ_SCAN_DONE   = const(6)
        
        self._ble = bluetooth.BLE()
            
        self._ble.active(True)
        self._ble.irq(self._irq)
        
        #self.aScan = []
        
    def _irq( self, event, data ):
        global isLocked
        global mapScan
        
        if isLocked == True :
            return
           
        isLocked = True
            
        if event == self._IRQ_SCAN_RESULT:
            addr_type, addr, adv_type, rssi, adv_data = data
            # the addr, adv_data, char_data, notify_data, and uuid entries in the tuples
            # are read-only memoryview instances pointing to bluetooth’s internal ringbuffer,
            # and are only valid during the invocation of the IRQ handler

            strAddress = binascii.hexlify(bytes(addr)).decode()

            if not strAddress in mapScan :
                mapScan[strAddress] = {}
            
            # 1st byte: length of the element (excluding the length byte itself)
            # 2nd byte: AD type – specifies what data is included in the element
            # AD data – one or more bytes - the meaning is defined by AD type
            
            payload = bytes(adv_data)
            
            mapScan[strAddress][ "address_type"    ] = addr_type
            mapScan[strAddress][ "adv_type"        ] = adv_type
            mapScan[strAddress][ "signal-strength" ] = rssi
            #mapScan[strAddress][ "payload"         ] = binascii.hexlify( payload ).decode()
            mapScan[strAddress][ "last-update"     ] = ticks_ms()
            
            iIndex  = 1
            advType = None
            
            while iIndex < len(payload):
                try:
                    data    = payload[iIndex+1: iIndex + payload[iIndex-1] ]
                    advType = payload[ iIndex ]
                    
                    mapScan[strAddress][ "advType" ] = advType
                    
                    if advType == self._ADV_TYPE_NAME :
                        mapScan[strAddress][ "advTypeName" ] = str(data, "utf-8")
                        
                    elif advType == self._ADV_TYPE_SHORT_NAME :
                        mapScan[strAddress][ "advTypeShortName" ] = str(data, "utf-8")
                        
                    elif advType == self._ADV_TYPE_UUID16_COMPLETE or advType == self._ADV_TYPE_UUID16_INCOMPLETE :
                        mapScan[strAddress][ "uuid16Complete" ] = bluetooth.UUID(struct.unpack("<h", data)[0])
                        
                    elif advType == self._ADV_TYPE_UUID32_COMPLETE or advType == self._ADV_TYPE_UUID32_INCOMPLETE :
                        mapScan[strAddress][ "uuid32Complete" ] = bluetooth.UUID(struct.unpack("<d", data)[0])
                        
                    elif advType == self._ADV_TYPE_UUID128_COMPLETE or advType == self._ADV_TYPE_UUID128_INCOMPLETE:
                        mapScan[strAddress][ "uuid128Complete" ] = bluetooth.UUID(data)
                        
                    elif advType == self._ADV_TYPE_EXTENSION :
                        id = data[1]<<8 | data[0]

                        mapScan[strAddress][ "manufacturer-id" ] = id
                        
                        
                        if id == 0x004C :
                            mapScan[strAddress][ "manufacturer-name"       ] = "Apple, Inc"
                            mapScan[strAddress][ "manufacturer-short-name" ] = "Apple"
                            
                        elif id == 0x00E0 :
                            mapScan[strAddress][ "manufacturer-name"       ] = "Google"
                            mapScan[strAddress][ "manufacturer-short-name" ] = "Google"
                            
                        elif id == 0x0006 :
                            mapScan[strAddress][ "manufacturer-name"       ] = "Microsoft"
                            mapScan[strAddress][ "manufacturer-short-name" ] = "MS"
                            
                        elif id == 0x0000 :
                            mapScan[strAddress][ "manufacturer-name"       ] = "Ericsson Technology Licensing"
                            mapScan[strAddress][ "manufacturer-short-name" ] = "Ericsson"
                            
                        elif id == 0x0001 :
                            mapScan[strAddress][ "manufacturer-name"       ] = "Nokia Mobile Phones"
                            mapScan[strAddress][ "manufacturer-short-name" ] = "Nokia"
                            
                        elif id == 0x0004 :
                            mapScan[strAddress][ "manufacturer-name"       ] = "Toshiba Corp."
                            mapScan[strAddress][ "manufacturer-short-name" ] = "Toshiba"
                            
                        elif id == 0x0003 :
                            mapScan[strAddress][ "manufacturer-name"       ] = "IBM Corp."
                            mapScan[strAddress][ "manufacturer-short-name" ] = "IBM"
                            
                        elif id == 0x0002 :
                            mapScan[strAddress][ "manufacturer-name"       ] = "Intel Corp."
                            mapScan[strAddress][ "manufacturer-short-name" ] = "Intel"
                            
                        elif id == 0x0075 :
                            mapScan[strAddress][ "manufacturer-name"       ] = "Samsung Electronics Co. Ltd."
                            mapScan[strAddress][ "manufacturer-short-name" ] = "Samsung"
                            
                        else:
                            if not "manufacturer-name" in mapScan[strAddress] :
                                mapScan[strAddress][ "manufacturer-name"       ] = "Unknown Manufacturer"
                                mapScan[strAddress][ "manufacturer-short-name" ] = "Unknown"

                except OSError as e:
                    aData.append( "ERROR " + e )
                iIndex += 1 + payload[iIndex-1]
                
        isLocked = False

    # Find devices
    def scan(self):
        self._ble.gap_scan(
            0,  # duration_ms: total scan duration: 0 means infinity
            30000, #interval_us = 30000, # scan every 30 mu-sec
            30000, # window_us   = 30000, # for scan-intervall: 30 mu-sec
            True   # active      = False 
        )
        
    def stop( self ):
        self._ble.gap_scan( None )
    
        
    
#====================================

import display
display = display.Display()
LINE_HEIGHT = 12

sniffer = BLESniffer()

import time

try:
    sniffer.scan()
    
    bAnimate = True
    
    while True:
        iSize = len( mapScan )
        
        display.clear()
        
        if iSize == 0 :
            display.text( "No Device found.", 0, 0 )
            display.text( "Scanning... {}".format( "\\" if bAnimate == True else "/" ), 0, LINE_HEIGHT )
            bAnimate = not bAnimate
            
        else :
            aDelItem = []
            
            for iIndex, strAddress in enumerate( mapScan ) :
                display.text( "{}/{} {} {}".format( iIndex+1, iSize, mapScan[strAddress][ "manufacturer-short-name" ], mapScan[strAddress][ "signal-strength" ] ), 0,  2*iIndex   *LINE_HEIGHT )
                display.text( "  {}".format( strAddress ), 0, (2*iIndex+1)*LINE_HEIGHT )
                iStrength = max( 0, min( 10, round((90 + mapScan[strAddress][ "signal-strength" ])/6)) )
                
                for iPosY in range( iStrength) :
                    display.hline( 0,  int((2*iIndex+1.5)*LINE_HEIGHT) - iPosY, 2+iPosY, 1 )
                
                if ( ticks_ms() - mapScan[strAddress][ "last-update" ] ) > 5000 :
                    aDelItem.append( strAddress )
                    
            for strAddress in aDelItem :
                try: del mapScan[strAddress]
                except KeyError: pass
        
        display.show()
        # print( mapScan )
        time.sleep( 0.1 )

#for key, value in dictA.items():
#   print (key, value)

except KeyboardInterrupt:
    pass

finally:
    sniffer.stop()

print( "Done." )

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.
....................................